Subset.py

In [2]:
import pandas as pd

def subset(file_paths=[]):

    dataframes = [pd.read_csv(file) for file in file_paths]
    combined_df = pd.concat(dataframes, ignore_index=True)

    return combined_df

Dataset.py

In [ ]:
from sklearn.model_selection import train_test_split
def dataset():
    file_paths = [
        'combined_features_with_date1.csv',
        'combined_features_with_date2.csv',
        'combined_features_with_date3.csv'
    ]

    Dataset = subset(file_paths=file_paths)

    # Step 4: Split the Dataset into Training and Testing data (80% training, 20% testing)
    TrainingData, TestData = train_test_split(Dataset, test_size=0.2, random_state=42)

    # Step 5: Export the datasets to CSV files
    TrainingData.to_csv("TrainingData.csv", index=False)
    TestData.to_csv("TestData.csv", index=False)

    with open("features_info.txt", "w") as f:
        f.write("Features used:\n")
        for feature in Dataset.columns:
            f.write(f"{feature}\n")

    print("Datasets have been successfully split and saved as 'TrainingData.csv' and 'TestData.csv'.")
    print("Features info saved as 'features_info.txt'.")

    return TrainingData, TestData


neuralnet.py

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def build_model(input_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_size,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1) 
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def train_nn(TrainingData, epochs=100, batch_size=32):
   
    X = TrainingData.drop(columns=["streamflow"]).values
    y = TrainingData["streamflow"].values


    scaler = StandardScaler()
    X = scaler.fit_transform(X)


    input_size = X.shape[1]
    model = build_model(input_size)

    # Train the model
    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1)
    print("Model training complete.")
    
    return model, scaler


if __name__ == "__main__":
    train_data, test_data = dataset()

    # Load training data
    NNModel, scaler = train_nn(train_data)